In [1]:
import os
import json
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

## Get parameter-group 1: FARM, location, crop name, yield, area

In [2]:
# data loader

# get farm data
def get_farm_data(farm_id):
    # dir = os.path.dirname(__file__)
    dir = os.getcwd()
    farm_data_path = os.path.join(dir, '../data/test/litefarm_test.csv')
    df = pd.read_csv(farm_data_path)
    
    farm = df.query(f"farm_id == '{farm_id}'")
    
    area = farm["area_in_m2"].iloc[0] * 0.0001  # Litefarm database use m^2 while holos calculation use ha
    crop_yield = farm["yield_kg"].iloc[0] / area  # kg/ha
    location = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
    crop_name = farm["common_crop_name"].iloc[0]
    year = farm["year"].iloc[0]
    
    return {'area': area, 'location': location, 'crop': crop_name, 'yield': crop_yield, 'year': year}

In [3]:
# test
farm_id = '0369f026-1f90-11ee-b788-0242ac150004'
get_farm_data(farm_id)

{'area': 0.1409,
 'location': [<POINT (-71.519 46.476)>],
 'crop': 'Soybean',
 'yield': 153158.2682753726,
 'year': 2021}

## Get parameter-group 2: MODIFIERS / reduction factors, RF_*

In [4]:
def add_province():
    # get the province base on the farm location
    farm_data = get_farm_data(farm_id)
    farm_point = gpd.GeoDataFrame({'geometry': farm_data["location"]}, crs="EPSG:4326")
        
    dir = os.getcwd()
    province_shp_path = os.path.join(dir, '../data/external/province_500m')
    provinces = gpd.read_file(province_shp_path)
    provinces = provinces.to_crs("EPSG:4326") 

    farm_province = gpd.sjoin(farm_point, 
                         provinces[["PRENAME", "geometry"]], 
                         how='left', 
                         predicate='within').drop(columns=['index_right'])
    
    farm_province.rename(columns={'PRENAME': 'province'}, inplace=True)
    
    return farm_province

In [5]:
# test
add_province()

,geometry,province
0,POINT (-71.51895 46.47619),Quebec


In [6]:
def get_province():
    province = add_province()["province"].iloc[0]
    return province

In [7]:
# TEST
get_province()

'Quebec'

In [8]:
def get_region():
    province = get_province()
    western_canada = [
        'Alberta', 'British Columbia', 'Manitoba', 
        'Saskatchewan', 'Northwest Territories', 'Nunavut'
    ]
    
    if province in western_canada:
        return 'western_canada'
    else:
        return 'eastern_canada'

In [9]:
# test
get_region()

'eastern_canada'

In [10]:
def get_modifiers(rf_am='default', rf_cs='Annual', rf_ns='RF_NS_CRN', tillage='unknown', soil_texture='unknown'):
    # get reduction factors, RF_*
    # reduction factors: coefficient used to estimate the decrease in N2O 
    # emissions resulting from specific management practices compared to a baseline scenario. 
    region = get_region()
    
    dir = os.getcwd()
    RF_AM_path = os.path.join(dir, '../data/preprocessed/modifier_rf_am.csv')
    RF_CS_path = os.path.join(dir, '../data/preprocessed/modifier_rf_cs.csv')
    RF_NS_path = os.path.join(dir, '../data/preprocessed/modifier_rf_ns.csv')
    RF_Till_path = os.path.join(dir, '../data/preprocessed/modifier_rf_till.csv')
    RF_TX_path = os.path.join(dir, '../data/preprocessed/modifier_rf_tx.csv')

    RF_AM_df = pd.read_csv(RF_AM_path)
    RF_CS_df = pd.read_csv(RF_CS_path)
    RF_NS_df = pd.read_csv(RF_NS_path)
    RF_Till_df = pd.read_csv(RF_Till_path)
    RF_TX_df = pd.read_csv(RF_TX_path)

    # RF_AM, application method, LiteFarm data do not have this field, use default 1.
    RF_AM = RF_AM_df.query(f"method == '{rf_am}'")['value'].iloc[0]

    # RF_CS, currently the calculation method only involves annual, thus use 'Annual' by default, 1
    RF_CS = RF_CS_df.query(f"group == '{rf_cs}'")['value'].iloc[0]

    # RF_NS, current calculation only involves crop residue nitrogen, thus only RF_NS_CRN is used
    RF_NS = RF_NS_df.query(f"N_source == '{rf_ns}'")['value'].iloc[0]

    # RF_Till, by default set as 1 "unknown", LiteFarm data do not provide this information
    RF_Till = RF_Till_df.query(f"region=='{region}' & tillage == '{tillage}'")['value'].iloc[0]

    # RF_TX
    # here should link to Ftopo data, need to do!
    RF_TX = RF_TX_df.query(f"region=='{region}' & soil_texture == '{soil_texture}'")['value'].iloc[0]

    return {"RF_AM": RF_AM, "RF_CS": RF_CS, "RF_NS": RF_NS, "RF_Till": RF_Till, "RF_TX": RF_TX}

In [11]:
# test
get_modifiers()

{'RF_AM': 1.0, 'RF_CS': 1.0, 'RF_NS': 0.84, 'RF_Till': 1.0, 'RF_TX': 1.0}

## Get parameter-group 3: CROP GROUP parameters: Carbon_concentration, S_s, S_r, S_p

In [18]:
# def map_simplified_crop_name():
    # LiteFarm has a flexible protocol for crop names, 
    # thus, map crop name to simplified version???
    # need to figure out how to do this?
    # or this should be something that LiteFarm team do
    # for now, we assume the crop name is clean and ready-to-use.
    

IndentationError: expected an indented block (4272965401.py, line 7)

In [12]:
def get_crop_group():
    # determine the group of crops: annual, perennial, cover, silage, root.
    farm_data = get_farm_data(farm_id)
    crop = farm_data['crop'].lower()

    dir = os.getcwd()
    crop_to_group_map_path = os.path.join(dir, '../data/preprocessed/crop_to_group.csv')
    crop_to_group_map_df = pd.read_csv(crop_to_group_map_path)

    crop_group = crop_to_group_map_df.query(f"crop == '{crop}'")['group'].iloc[0]

    return crop_group

In [13]:
# test
get_crop_group()

'Annual'

In [14]:
def get_crop_group_parameters():
    # get crop-group related parameters: Carbon_concentration, S_s, S_r, S_p
    crop_group = get_crop_group()

    dir = os.getcwd()
    crop_group_params_path = os.path.join(dir, '../data/preprocessed/crop_group_parameters.csv')
    crop_group_params_df = pd.read_csv(crop_group_params_path)

    crop_group_params = crop_group_params_df[crop_group_params_df['group'] == crop_group].iloc[0].to_dict()
    return crop_group_params

In [15]:
# test
get_crop_group_parameters()

{'group': 'Annual',
 'carbon_concentration': 0.45,
 'S_s': 100,
 'S_r': 100,
 'S_p': 2}

## Get parameter-group 4: CROP parameters: N_*, and R_*

In [16]:
def get_crop_parameters():
    # get crop specific parameters: N_p, N_s, N_r, N_e, R_p, R_s, R_r, R_e
    farm_data = get_farm_data(farm_id)
    crop = farm_data['crop']

    dir = os.getcwd()
    crop_params_path = os.path.join(dir, '../data/preprocessed/crop_parameters.csv')
    crop_params_df = pd.read_csv(crop_params_path)
    crop_params_df.head()

    crop_params = crop_params_df[crop_params_df['crop'] == crop].iloc[0].to_dict()
    crop_params.pop('group', None) 
    crop_params.pop('holos_crop_name', None)
    
    return crop_params

In [17]:
# test
get_crop_parameters()

{'crop': 'Soybean',
 'condition': 'Canada',
 'moisture': 14,
 'R_p': 0.304,
 'R_s': 0.455,
 'R_r': 0.146,
 'R_e': 0.095,
 'N_p': 67.0,
 'N_s': 6.0,
 'N_r': 10.0,
 'N_e': 10.0}

## Get parameter-group 5: CLIMATE parameters, P, PE, and Ftopo

In [18]:
def get_climate_parameters():
    # currently, we use the default Holos data derived from SLC
    add_province()

In [19]:
gdf = add_province()
gdf

,geometry,province
0,POINT (-71.51895 46.47619),Quebec


In [22]:
dir = os.getcwd()
default_climate_path = os.path.join(dir, '../data/raw/Holos/ecodistrict_to_ecozone_mapping.csv')
slc_polygons_path = os.path.join(dir, '../data/external/slc')
default_climate_df = pd.read_csv(default_climate_path)
slc_polygons = gpd.read_file(slc_polygons_path)
slc_polygons = slc_polygons.set_crs('EPSG:4269', inplace=True)
slc_polygons = slc_polygons.to_crs('EPSG:4326')

In [23]:
default_climate_df.head()

,Ecodistrict,Ecozone,Province,PMayToOct,PEMayToOct,Ftopo,SoilType,SoilTexture
0,358,Boreal Shield West,Manitoba,348,453,0.00,Brown Chernozem,Coarse
1,358,Boreal Shield West,Saskatchewan,348,453,0.00,Brown Chernozem,Coarse
2,371,Boreal Shield West,Manitoba,376,581,0.33,Black/Gray Chernozem,Fine
3,371,Boreal Shield West,Ontario,376,581,0.33,Eastern Canada,Fine
4,375,Boreal Shield West,Manitoba,411,579,0.33,Black/Gray Chernozem,Fine


In [24]:
slc_polygons.head()

,AREA,PERIMETER,POLY_ID,ECO_ID,geometry
0,1.376666,6.159230,14003,14,"POLYGON ((-73.74820 83.60566, -72.61377 83.626..."
1,1.580340,9.611541,14002,14,"POLYGON ((-70.91842 83.63315, -69.65201 83.612..."
2,0.932502,5.332157,14004,14,"POLYGON ((-75.75444 83.56109, -74.13673 83.597..."
3,2.408007,13.902721,14005,14,"POLYGON ((-77.61654 83.51138, -76.16966 83.550..."
4,3.511283,27.056163,14001,14,"POLYGON ((-67.62450 83.54608, -65.74776 83.467..."


In [25]:
farm_polygons = gpd.sjoin(farm_gdf, 
                          slc_polygons[["POLY_ID", "ECO_ID", "geometry"]],
                          how='left', 
                          predicate='within'
                         ).drop(columns=['index_right'])
farm_polygons.head()

,geometry,province,POLY_ID,ECO_ID
0,POINT (-71.51895 46.47619),Quebec,540011,540


In [26]:
farm_ecodistrict_climate = pd.merge(farm_polygons, default_climate_df,
                                    how='left',
                                    left_on=['ECO_ID','province'],
                                    right_on = ['Ecodistrict','Province'])
farm_ecodistrict_climate

,geometry,province,POLY_ID,ECO_ID,Ecodistrict,Ecozone,Province,PMayToOct,PEMayToOct,Ftopo,SoilType,SoilTexture
0,POINT (-71.51895 46.47619),Quebec,540011,540,540,Mixedwood Plains,Quebec,652,556,11.71,Eastern Canada,Coarse


In [27]:
farm_ecodistrict_climate = farm_ecodistrict_climate.drop(columns=['ECO_ID', 'Ecozone', 'province', 'SoilType'])
farm_ecodistrict_climate

,geometry,POLY_ID,Ecodistrict,Province,PMayToOct,PEMayToOct,Ftopo,SoilTexture
0,POINT (-71.51895 46.47619),540011,540,Quebec,652,556,11.71,Coarse


In [28]:
soil_texture = farm_ecodistrict_climate["SoilTexture"].iloc[0]
soil_texture

'Coarse'

In [29]:
P_i = farm_ecodistrict_climate["PMayToOct"].iloc[0]
P_i

652

In [30]:
PE_i = farm_ecodistrict_climate["PEMayToOct"].iloc[0]
PE_i

556

In [31]:
Ftopo = farm_ecodistrict_climate["Ftopo"].iloc[0]
Ftopo

11.71

In [32]:
climate_dict = {"soil_texture": soil_texture, "P": P_i, "PE": PE_i, "FR_Topo": Ftopo}
climate_dict

{'soil_texture': 'Coarse', 'P': 652, 'PE': 556, 'FR_Topo': 11.71}